read from results_0 to get 50 initial points

In [1]:
import pandas as pd
import numpy as np
import math

file_name = "results.csv"
X_data = pd.read_csv(file_name, usecols=[0, 1, 2])
X_init = X_data.values
print(X_init.shape)

(21, 3)


In [2]:
from imp.imp_postprocessing import calculate_fdisper, get_file_list
from tgv.tgv_postprocessing import calculate_ftke, calculate_fcons

def get_Y(para):

    ## imp
    target_folder_name_imp = f"imp/results/fullimplosion_{str(para[0])}_{str(para[1])}_{str(para[2])}"
    test_file = get_file_list(f"{target_folder_name_imp}/domain/data_*.h5")
    if test_file.endswith('nan.h5'):
        fdisper = 9
    else:
        fdisper = calculate_fdisper(target_folder_name_imp)
        fdisper = np.round(fdisper, 4)

    ## tgv
    target_folder_name_tgv = f"tgv/results/tgv_{str(para[0])}_{str(para[1])}_{str(para[2])}"
    # tgv_tke
    ftke = calculate_ftke(f"{target_folder_name_tgv}/domain/data_*.h5")
    ftke = np.round(ftke, 4)
    if math.isnan(ftke):
        ftke = 4
    
    # tgv_cons
    fcons = calculate_fcons(f"{target_folder_name_tgv}/domain/data_*.h5")
    fcons = np.round(fcons, 4)
    if math.isnan(fcons):
        fcons = 1
    return np.array([-fdisper, -ftke, -fcons])
    

In [3]:
Y_sample_multi = np.array([get_Y(para) for para in X_init])
# reshape
n_samples = Y_sample_multi.shape[0]
n_f = Y_sample_multi.shape[1] # number of functions
Y_sample_multi = Y_sample_multi.T.reshape(n_f, n_samples, 1) # [n_f, n_samples, 1]

Load imp/results/fullimplosion_0.8585_0.7218_0.3537/domain/data_2.5003803026.h5
Load tgv/results/tgv_0.8585_0.7218_0.3537/domain/data_10.0001551040.h5
Load tgv/results/tgv_0.8585_0.7218_0.3537/domain/data_10.0001551040.h5
(64, 64, 64)
Load tgv/results/tgv_0.8585_0.7218_0.3537/domain/data_10.0001551040.h5
(64, 64, 64)
Load tgv/results/tgv_0.8585_0.7218_0.3537/domain/data_10.0001551040.h5
(64, 64, 64)
Load tgv/results/tgv_0.8585_0.7218_0.3537/domain/data_10.0001551040.h5
(64, 64, 64)
19.491135075699486
Load imp/results/fullimplosion_0.5569_0.3671_0.82/domain/data_2.5005509979.h5
Load tgv/results/tgv_0.5569_0.3671_0.82/domain/data_nan.h5
Load tgv/results/tgv_0.5569_0.3671_0.82/domain/data_nan.h5
(64, 64, 64)
Load tgv/results/tgv_0.5569_0.3671_0.82/domain/data_nan.h5
(64, 64, 64)
Load tgv/results/tgv_0.5569_0.3671_0.82/domain/data_nan.h5
(64, 64, 64)
Load tgv/results/tgv_0.5569_0.3671_0.82/domain/data_nan.h5
(64, 64, 64)
nan
Load imp/results/fullimplosion_0.5006_0.5842_0.8103/domain/data_2

In [4]:
print(Y_sample_multi.shape)

(3, 21, 1)


In [5]:
Y_reshaped = Y_sample_multi.reshape(3, 21).T 
combined = np.hstack((X_init, Y_reshaped))
df = pd.DataFrame(combined, columns=['eta_eno','eta_v','ducros_cutoff','fdisper','ftke','fcons'])
csv_file_name = 'initial_points.csv'
df.to_csv(csv_file_name, index=False)